<a href="https://colab.research.google.com/github/subhalingamd/btp-hate-speech-profiling/blob/btp-final/our_model/our_model_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
ROOT_PATH = '/content/drive/MyDrive/btp'

def _path(path):
  return f'{ROOT_PATH}/{path}'

# Imports

In [ ]:
# !wget https://nlp.stanford.edu/data/glove.twitter.27B.zip

In [ ]:
# !pip install -q pytorch-lightning 
# !pip install -q bpemb  --no-deps
# !pip install -q sentencepiece
# !pip install -q transformers

In [ ]:
# import torch
# import torch.nn as nn
# import torch.nn.functional as F
# import torch.optim as optim
# from torch.utils.data import DataLoader, Dataset
# from transformers import get_scheduler, AdamW

# from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence


# import torchtext
# from torchtext import datasets, vocab
# from torchtext.legacy import data as textdata
## ???from torchtext.vocab import GloVe

import numpy as np
import pandas as pd

# import pytorch_lightning as pl
# from pytorch_lightning import Trainer
# from pytorch_lightning import LightningModule
# from pytorch_lightning.callbacks.early_stopping import EarlyStopping
# from pytorch_lightning.callbacks.model_checkpoint import ModelCheckpoint

# from bpemb import BPEmb

import random
import os

from argparse import ArgumentParser

import pickle

import string
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# from pytorch_lightning.utilities.seed import seed_everything


from tqdm import tqdm
from sklearn.linear_model import LogisticRegression as LR
from sklearn.svm import SVC as SVM
from sklearn.ensemble import RandomForestClassifier as RF
from xgboost import XGBClassifier as XGB
import lightgbm as lgb
from sklearn.naive_bayes import MultinomialNB


In [ ]:
import nltk
nltk.download(['punkt', 'wordnet', 'stopwords'])
nltk.download(['averaged_perceptron_tagger', 'universal_tagset'])
nltk.download('all')
from nltk.corpus import sentiwordnet as swn
from nltk.tag import pos_tag,map_tag
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords

pstem = PorterStemmer()
lem = WordNetLemmatizer()
stop_words = stopwords.words('english')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.
[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...


In [ ]:
import gensim.downloader as api

In [ ]:
def seed_everything(seed=42):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    # torch.manual_seed(seed)
    # torch.cuda.manual_seed(seed)
    # torch.cuda.manual_seed_all(seed)
    # torch.backends.cudnn.deterministic = True
    # torch.backends.cudnn.benchmark = False

seed_everything(42)

# Args

In [ ]:
def parse_args():
	parser = ArgumentParser(description='Word Meaning Comparison')
	parser.add_argument('--data_train', type=str, default=_path('inputs/data_pan21_train_en.tsv'))
	parser.add_argument('--data_test', type=str, default=_path('inputs/data_pan21_test_en.tsv'))
	parser.add_argument('--embedding', type=str, default="glove-twitter-25", 
	                    choices=["glove-twitter-200", "glove-twitter-100", "glove-twitter-50", "glove-twitter-25"])
	
	parser.add_argument('--gpu', type=int, default=1)
	parser.add_argument('--batch_size', type=int, default=4)
	parser.add_argument('--epochs', type=int, default=25)
	parser.add_argument('--lr', type=float, default=0.001)
	parser.add_argument('--lr_decay', type=float, default=0.0005)	# removed

	parser.add_argument('--rnn_hidden_dim', type=int, default=8)
	parser.add_argument('--num_rnn_layers', type=int, default=1)
	parser.add_argument('--dropout', type=float, default=0.5)

	return parser.parse_known_args()[0]


args = parse_args()
print(args)

Namespace(batch_size=4, data_test='/content/drive/MyDrive/btp/inputs/data_pan21_test_en.tsv', data_train='/content/drive/MyDrive/btp/inputs/data_pan21_train_en.tsv', dropout=0.5, embedding='glove-twitter-25', epochs=25, gpu=1, lr=0.001, lr_decay=0.0005, num_rnn_layers=1, rnn_hidden_dim=8)


In [ ]:
word_embedding = api.load(args.embedding)
args.embedding_dim = word_embedding['word'].shape[0]

[==================================================] 100.0% 104.8/104.8MB downloaded


In [ ]:
args.embedding_dim = word_embedding['word'].shape[0]

# Main

In [ ]:
def remove_punctuations(text):
    text = text.replace('##emoji##', '').replace('#url#', '').replace('#user#', '').replace('##rt##', '')#.replace('#hashtag#','')
    exclist = string.punctuation #+ string.digits
    table = str.maketrans(exclist, ' '*len(exclist))
    return ' '.join(text.translate(table).split())

def clean_df(df):
  for col in [x for x in df.columns if x != 'label']:
    df[col] = df[col].apply(remove_punctuations)
  return df

In [ ]:
df_train = pd.read_csv(args.data_train, sep='\t')
df_test = pd.read_csv(args.data_test, sep='\t')

In [ ]:
# df_train, df_val = train_test_split(df_train_,
#                                     stratify=df_train_['label'].tolist(), 
#                                     test_size=0.1,
#                                     shuffle=True,
#                                     random_state=42,
#                                     )

In [ ]:
df_train.head()

,tweet_1,tweet_2,tweet_3,tweet_4,tweet_5,tweet_6,tweet_7,tweet_8,tweet_9,tweet_10,...,tweet_192,tweet_193,tweet_194,tweet_195,tweet_196,tweet_197,tweet_198,tweet_199,tweet_200,label
0,##rt## funny how 15 days to slow the spread t...,##rt## why did minneapolis just give george f...,##rt## to be fair he has done a lot of underc...,##rt## president trump got us the #hashtag# v...,##rt## is the case against former officer der...,##rt## the fbi cia and military now feel tota...,##rt## biden bashed for taking all credit for...,##rt## bidens disappointing prime-time speech...,##rt## cuomo shouldnt be worried about resign...,##rt## breaking family of george floyd settle...,...,##rt## why isnt cuomos deadly coronavirus cov...,##rt## can we get more information about this...,##rt## raise your hand if you wont be wearing...,##rt## which is precisely why the media did w...,##rt## poll one in six biden voters would hav...,##rt## i just spit out my water #url#,##rt## because its cracked pipes he would be ...,##rt## donald trump is back on the record – a...,##rt## trump showed republicans how to help m...,0
1,if you havent tried parler yet please give it ...,##emoji## ##emoji## my president ##emoji## ##...,#user# my president ##emoji## ##emoji## bump,#user# i did election night newsmax for me ##...,i ##emoji## imagine that ##emoji## confidentia...,##emoji## ##emoji## lets show usa today who w...,#user# ##emoji## ##emoji## get well soon sir ...,we are all #hashtag# for you and our beautiful...,##rt## #user# do you support our potus (pleas...,#user# law and order thank you potus,...,anybody would be nuts to vote for this guy ##e...,##rt## rt if its time institute nationwide vo...,more ppl need to hear this doctor tell it like...,##emoji## truth ##emoji## #url#,dream on honey ##emoji## ##emoji## ##emoji## #...,i so want to see her and bill face justice #url#,i hope so@ #url#,##rt## deplorables mount up ##emoji## ##emoji...,#user# im with ya #hashtag# #hashtag#,0
2,##rt## disagreement with our scholarship is n...,##rt## today is the 52nd day of the biden adm...,##rt## the cartels control the us-mexico bord...,##rt## and again clearly the federal courthou...,#user# the california mom who did the same th...,yes #hashtag# and youre a hateful bigot if you...,#user# #user# #user# blew her claim out of th...,#user# #user# #user# #user# ooh my favorite d...,##rt## woke christian asks jesus to return al...,##rt## depends on the training #url#,...,##rt## this is a l you posted cringe you just...,##rt## remember my wise friend we caught up t...,##rt## correct judge (correctly noted that am...,##rt## joe biden opened himself up to questio...,##rt## warning this thread may make you sick ...,youre never gonna unsee that you can thank me ...,but why is nicole kidman dressed as a fruit ro...,#user# according to the theory on allergies s...,##rt## the new total is 1927000 signatures,0
3,#user# #user# tell people what to do hmm that...,##rt## #user# i guarantee the signature rejec...,##rt## its easy to be woke when youve never g...,#user# imagine how many handguns can $14m get...,#user# victims of black crimes too just like ...,#user# #user# trump is a fucking fail preside...,i just edited this headline story on #hashtag#...,harvard = trash #url#,##rt## heres to all the dads doing their best...,##rt## welcome to california the politicians ...,...,##rt## i copied this from parler it is so goo...,#user# #user# where is the evidence that he i...,whats hunter bidens next job after dad loses,#user# vote for trump,##rt## air force veteran aja smith hopes to b...,i love hunter biden he is fucking up his dads ...,who got the worse brother,##rt## suspending white house officials twitt...,#user# #user# all the republican majority are...,0
4,#user# none at all just the ability to lie an...,#user# actually most seem to be communists wh...,#user# spot on theyd both bare their arses on...,#user# with your media and government the ver...,#user# if there is a god god willing he

In [ ]:
df_train = clean_df(df_train)
df_test = clean_df(df_test)

In [ ]:
df_train.head()

,tweet_1,tweet_2,tweet_3,tweet_4,tweet_5,tweet_6,tweet_7,tweet_8,tweet_9,tweet_10,...,tweet_192,tweet_193,tweet_194,tweet_195,tweet_196,tweet_197,tweet_198,tweet_199,tweet_200,label
0,funny how 15 days to slow the spread turned in...,why did minneapolis just give george floyds fa...,to be fair he has done a lot of undercover wor...,president trump got us the hashtag vaccine pre...,is the case against former officer derek chauv...,the fbi cia and military now feel totally comf...,biden bashed for taking all credit for covid 1...,bidens disappointing prime time speech was a c...,cuomo shouldnt be worried about resigning he s...,breaking family of george floyd settle wrongfu...,...,why isnt cuomos deadly coronavirus cover up th...,can we get more information about this speaker...,raise your hand if you wont be wearing a mask ...,which is precisely why the media did whatever ...,poll one in six biden voters would have change...,i just spit out my water,because its cracked pipes he would be dealing ...,donald trump is back on the record – asked abo...,trump showed republicans how to help minority ...,0
1,if you havent tried parler yet please give it ...,my president,my president bump,i did election night newsmax for me,i imagine that confidential documents obtained...,lets show usa today who won the debate,get well soon sir we love you,we are all hashtag for you and our beautiful h...,do you support our potus please help by retwee...,law and order thank you potus,...,anybody would be nuts to vote for this guy,rt if its time institute nationwide voter id f...,more ppl need to hear this doctor tell it like...,truth,dream on honey,i so want to see her and bill face justice,i hope so,deplorables mount up,im with ya hashtag hashtag,0
2,disagreement with our scholarship is not possi...,today is the 52nd day of the biden administrat...,the cartels control the us mexico border on ru...,and again clearly the federal courthouse in po...,the california mom who did the same thing is o...,yes hashtag and youre a hateful bigot if you t...,blew her claim out of the water anyway it was ...,ooh my favorite dessert,woke christian asks jesus to return all the sh...,depends on the training,...,this is a l you posted cringe you just admitte...,remember my wise friend we caught up today,correct judge correctly noted that am referenc...,joe biden opened himself up to questions from ...,warning this thread may make you sick,youre never gonna unsee that you can thank me ...,but why is nicole kidman dressed as a fruit ro...,according to the theory on allergies some of m...,the new total is 1927000 signatures,0
3,tell people what to do hmm that sounds communi...,i guarantee the signature reject rate will be ...,its easy to be woke when youve never gotten ju...,imagine how many handguns can 14m get for asia...,victims of black crimes too just like the othe...,trump is a fucking fail president he placed th...,i just edited this headline story on hashtag y...,harvard trash,heres to all the dads doing their best,welcome to california the politicians that run...,...,i copied this from parler it is so good and th...,where is the evidence that he is a white supre...,whats hunter bidens next job after dad loses,vote for trump,air force veteran aja smith hopes to be first ...,i love hunter biden he is fucking up his dads ...,who got the worse brother,suspending white house officials twitter accou...,all the republican majority areas be shut down,0
4,none at all just the ability to lie and adore ...,actually most seem to be communists which as t...,spot on theyd both bare their arses on holywoo...,with your media and government the very talk o...,if there is a god god willing he will do no su...,surely a careless injection is not going to be...,im shocked i tell you shocked,with an appointment taking 3 weeks i doubt he ...,i saw a tv advert with a white person in it th...,wow not only are you a fantastic artist youre ...,...,how is the press racist the better question

# Dataset

In [ ]:
# tk = nltk.TweetTokenizer()
def process_dataset(data, embedding, pooling='mean'):
    assert len(data) > 0, 'data must be a non-empty list'
    unk_count, unk_emb_count, unk_swn_count = 0, 0, 0
    def _get_swn_score(s, embedding):
        unk_count=0
        pos, neut, neg = np.zeros(args.embedding_dim), np.zeros(args.embedding_dim), np.zeros(args.embedding_dim)
        norm_pos, norm_neut, norm_neg = 0, 0, 0
        ## s = tk.tokenize(s)
        ## tokens = [x for x in s if x not in stop_words]
        s = ' '.join([x for x in s.split() if x not in stop_words])
        tokens = s.split()
        tagged_sent = pos_tag(tokens)
        store_it = [(word, map_tag('en-ptb', 'universal', tag)) for word, tag in tagged_sent]
        #print("Tagged Parts of Speech:",store_it)

        for word,tag in store_it:
            if(tag=='NOUN'):
                tag='n'
            elif(tag=='VERB'):
                tag='v'
            elif(tag=='ADJ'):
                tag='a'
            elif(tag=='ADV'):
                tag = 'r'
            else:
                tag='nothing'
            
            if tag == 'nothing':
                unk_count += 1
                continue
            else:
                # if word in stop_words:
                #     continue
                concat = word+'.'+tag+'.01'
                try:
                    sent = swn.senti_synset(concat)
                    emb = embedding[word]
                    pos += sent.pos_score()*emb
                    neut += sent.obj_score()*emb
                    neg += sent.neg_score()*emb
                    norm_pos, norm_neut, norm_neg = norm_pos + sent.pos_score(), norm_neut + sent.obj_score(), norm_neg + sent.neg_score()
                    #print(word,tag,':',this_word_pos,this_word_neg)
                # except Exception as e:
                #     wor = lem.lemmatize(word)
                #     # if wor in stop_words:
                #     #     continue
                #     concat = wor+'.'+tag+'.01'
                #     # Checking if there's a possiblity of lemmatized word be accepted into SWN corpus
                #     try:
                #         sent = swn.senti_synset(concat)
                #         emb = embedding[wor]
                #         pos += sent.pos_score()*emb
                #         neut += sent.obj_score()*emb
                #         neg += sent.neg_score()*emb
                #         norm_pos, norm_neut, norm_neg = norm_pos + sent.pos_score(), norm_neut + sent.obj_score(), norm_neg + sent.neg_score()
                #     except Exception as e:
                #         wor = pstem.stem(word)
                #         # if wor in stop_words:
                #         #     continue
                #         concat = wor+'.'+tag+'.01'
                #         # Checking if there's a possiblity of stemmed word be accepted
                #         try:
                #             sent = swn.senti_synset(concat)
                #             emb = embedding[wor]
                #             pos += sent.pos_score()*emb
                #             neut += sent.obj_score()*emb
                #             neg += sent.neg_score()*emb
                #             norm_pos, norm_neut, norm_neg = norm_pos + sent.pos_score(), norm_neut + sent.obj_score(), norm_neg + sent.neg_score()
                #         except:
                #             unk_count += 1
                #             continue
                except:
                        unk_count += 1
                        continue
                            
        # print(unk_count, len(s.split()))
        # return (  pos/norm_pos if norm_pos != 0 else pos, 
        #           neut/norm_neut if norm_neut != 0 else neut,
        #           neg/norm_neg if norm_neg != 0 else neg  )
        return pos, neut, neg
        ##return [neg]

    def _get_embeddings(tweets, pooling, embedding):
        assert pooling in ("mean", "max", "min", "abs-max")
        
        result = np.stack(
            [np.concatenate(_get_swn_score(t, embedding=embedding), axis=0) 
                for t in tweets], 
            axis=0,
            )
        
        # print(result.shape)
        
        if pooling == 'mean':
            return np.mean(result, axis=0)
        elif pooling == 'max':
            return np.max(result, axis=0)
        elif pooling == 'min':
            return np.min(result, axis=0)
        elif pooling == 'abs-max':
            raise NotImplementedError
            # return result[
            #             torch.max(torch.abs(result), dim=0).indices, 
            #             list(range(result.size(1)))
            #             ]


    label_col = 'label'
    tweet_cols = [x for x in data.columns if x!=label_col]
    assert len(tweet_cols) == len(data.columns) - 1 == 200

    _labels = data[label_col].tolist()
    _tweets = data[tweet_cols].values.tolist()


    data_emb, data_label = [], []
    for label, tweets in zip(tqdm(_labels), _tweets):
        emb = _get_embeddings(tweets, pooling=pooling, embedding=embedding)
        # print(emb.shape)
        assert emb.shape[0] == 3*args.embedding_dim
        data_emb.append(emb)
        data_label.append(label)

    return data_emb, data_label


In [ ]:
# construct train, val, test dataset
train_dataset = process_dataset(df_train, embedding=word_embedding)
test_dataset = process_dataset(df_test, embedding=word_embedding)


100%|██████████| 100/100 [00:16<00:00,  6.22it/s]


In [ ]:

from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold

all_pipelines = {
    'svm': [('svm', SVM(
                                    # probability = True,
                                    verbose=0,
                                    max_iter=10000,
                                    random_state=42,
                                    )
                        ),
            ],
    'lr': [('lr', LR(
                                    verbose=0,
                                    max_iter=10000,
                                    random_state=42,
                                    )
                        ),],
    'rf': [('rf', RF(
                                    verbose=0,
                                    random_state=42,
                                    )
                        ),],
    'xgb': [('xgb', XGB(
                                    use_label_encoder=False,
                                    verbosity=0,
                                    random_state=42,
                                    )
                        ),],
    'lgb': [('lgb', lgb.LGBMClassifier(
                                    objective = 'binary',
                                    verbose=0,
                                    random_state=42,
                                    deterministic=True,
                                    early_stopping_rounds=200,
                                    )
                        ),],
}

all_parameters = {
    'svm': {
            'svm__C': [.1, .2, .5, 1, 2, 5, 10, 20, 50, 100],
            'svm__kernel': ['linear', 'rbf'], # ['rbf', 'linear', 'poly', 'sigmoid']
            # 'svm__gamma': [0.001, 0.01, 0.1, 1, 10, 100],
        },
    'lr': {
            'lr__C': [.1, .2, .5, 1, 2, 5, 10, 20, 50, 100],
            'lr__penalty': ['l1', 'l2'],
            'lr__solver': ['liblinear', 'saga'], # ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']
        },
    'rf': {
            'rf__n_estimators': [20, 50, 100, 200, 500], # try -> [50, 100, 200, 500]
            'rf__max_depth': [None, 5, 10, 50, 100],
            'rf__min_samples_leaf':  [1, 2, 4],
            'rf__max_features': ['auto'], # ['auto', 'sqrt', 'log2', None]
            'rf__bootstrap': [True], # [True, False]
            'rf__oob_score': [True], # [True, False]
        },
    'xgb': {
            'xgb__n_estimators': [100, 200, 300],
            'xgb__max_depth': [3, 5, 7],
            'xgb__eta': [0.05, 0.1, 0.3],
            "xgb__subsample": [0.6, 0.8, 1.0],
            "xgb__colsample_bytree": [0.6, 0.8, 1.0],
        },
    'lgb': {
            'lgb__boosting_type' : ['dart', 'gbdt'], # dart is good
            'lgb__n_estimators': [50, 100, 200],
            'lgb__learning_rate': [0.05, 0.1, 0.3],

            'lgb__subsample': [0.6, 0.8, 1.0],

            'lgb__colsample_bytree' : [0.6, 0.8, 1.0],
            # 'lgb__l1_regularization' : [1,1.2],
            # 'lgb__l2_regularization' : [1,1.2,1.4],
            # 'lgb__num_leaves': [6, 8, 10,12, 16], # large num_leaves helps improve accuracy but might lead to over-fitting
        },
}


In [ ]:
for model in ('svm', 'lr', 'rf', 'lgb', 'xgb', ):
    print(model, '>'*(32-len(model)))
    pipeline = Pipeline(all_pipelines[model])
    parameters = all_parameters[model]
    
    grid_search = GridSearchCV(
                            estimator = pipeline, 
                            param_grid = parameters, 
                            cv = StratifiedKFold(10, shuffle=True, random_state=42),
                            scoring = 'accuracy',
                            n_jobs = -1,
                            return_train_score = True,
                            verbose = 4,
                        )


    # x = grid_search.fit(train_dataset[0], train_dataset[1])
    # z = grid_search.predict(test_dataset[0])

    x = grid_search.fit(np.stack(train_dataset[0]), train_dataset[1])
    z = grid_search.predict(np.stack(test_dataset[0]))

    print(classification_report(test_dataset[1], z, digits=4))

    print(grid_search.best_params_)

    print('<'*32)

svm >>>>>>>>>>>>>>>>>>>>>>>>>>>>>
Fitting 10 folds for each of 20 candidates, totalling 200 fits
              precision    recall  f1-score   support

           0     0.7347    0.7200    0.7273        50
           1     0.7255    0.7400    0.7327        50

    accuracy                         0.7300       100
   macro avg     0.7301    0.7300    0.7300       100
weighted avg     0.7301    0.7300    0.7300       100

{'svm__C': 0.5, 'svm__kernel': 'linear'}
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
lr >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
Fitting 10 folds for each of 40 candidates, totalling 400 fits
              precision    recall  f1-score   support

           0     0.7826    0.7200    0.7500        50
           1     0.7407    0.8000    0.7692        50

    accuracy                         0.7600       100
   macro avg     0.7617    0.7600    0.7596       100
weighted avg     0.7617    0.7600    0.7596       100

{'lr__C': 1, 'lr__penalty': 'l1', 'lr__solver': 'liblinear'}
<<<<<<<<<<<<<<<<<

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
810 fits failed out of a total of 1620.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
810 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/pipeline.py", line 394, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/usr/local/lib/python3.7/dist-packages/lightgbm/sklearn.py", line 744, in fit
    callbacks=callbacks)
  File "/usr/local/lib/python3.7/dist-packages/lightgbm/

              precision    recall  f1-score   support

           0     0.7021    0.6600    0.6804        50
           1     0.6792    0.7200    0.6990        50

    accuracy                         0.6900       100
   macro avg     0.6907    0.6900    0.6897       100
weighted avg     0.6907    0.6900    0.6897       100

{'lgb__boosting_type': 'dart', 'lgb__colsample_bytree': 0.8, 'lgb__learning_rate': 0.1, 'lgb__n_estimators': 200, 'lgb__subsample': 0.6}
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
xgb >>>>>>>>>>>>>>>>>>>>>>>>>>>>>
Fitting 10 folds for each of 243 candidates, totalling 2430 fits
              precision    recall  f1-score   support

           0     0.6596    0.6200    0.6392        50
           1     0.6415    0.6800    0.6602        50

    accuracy                         0.6500       100
   macro avg     0.6505    0.6500    0.6497       100
weighted avg     0.6505    0.6500    0.6497       100

{'xgb__colsample_bytree': 0.6, 'xgb__eta': 0.05, 'xgb__max_depth': 5, 'xgb__n

In [ ]:
# grid_search.best_params_

In [ ]:
from sklearn.neural_network import MLPClassifier

In [ ]:
pipeline = Pipeline([
                     ('nn', MLPClassifier(
                                max_iter=10000, 
                                solver="lbfgs",
                                random_state=42, 
                                )
                     )
])

parameters = {
    'nn__hidden_layer_sizes': [ (16,), (8,), (4,), (16, 8), (8, 4), (16, 8, 4),  ],
    'nn__activation': ['tanh', 'relu', 'identity'],
}

grid_search = GridSearchCV(
                            estimator = pipeline, 
                            param_grid = parameters, 
                            cv = StratifiedKFold(10, shuffle=True, random_state=42),
                            scoring = 'accuracy',
                            n_jobs = -1,
                            return_train_score = True,
                            verbose = 4,
                        )


x = grid_search.fit(train_dataset[0], train_dataset[1])
z = grid_search.predict(test_dataset[0])
print(classification_report(test_dataset[1], z, digits=4))

print(grid_search.best_params_)

Fitting 10 folds for each of 18 candidates, totalling 180 fits
              precision    recall  f1-score   support

           0     0.6042    0.5800    0.5918        50
           1     0.5962    0.6200    0.6078        50

    accuracy                         0.6000       100
   macro avg     0.6002    0.6000    0.5998       100
weighted avg     0.6002    0.6000    0.5998       100

{'nn__activation': 'tanh', 'nn__hidden_layer_sizes': (16, 8, 4)}
